In [15]:
#Importing Packages
import sys
sys.path.append("..")
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
import pandas as pd
import numpy as np
import timeit
import time
from glove.glovevectorizer import GloveVectorizer

In [2]:
#Train & Test
train = pd.read_csv('C:/Users/franc/OneDrive/Documentos/Projects/r52/tclass/Data/train/reuters_train_52/r52-train-all-terms.txt', header=None, sep='\t')
test = pd.read_csv('C:/Users/franc/OneDrive/Documentos/Projects/r52/tclass/Data/test/reuters_test_52/r52-test-all-terms.txt', header=None, sep='\t')
#Name the features
train.columns = ['label', 'content']
test.columns = ['label', 'content']

In [3]:
xg_reg=xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.3, learning_rate=0.1,max_depth=5, alpha=10, n_estimators=10)

In [4]:
trainLabelLst = list(dict.fromkeys(train.label))
#trainLabelLst

In [5]:
def get_label(label_str_lst, label_lst):
    return_lst = []
    for s1 in label_str_lst:
        for i, label in enumerate(label_lst):
            if s1 == label:
                return_lst.append(i)
    return return_lst

In [6]:
vectorizer = GloveVectorizer()
Xtrain = vectorizer.fit_transform(train.content) # get wordvectors

# get label index
train['label_idx'] = get_label(train['label'],trainLabelLst)
Ytrain = train.label_idx

data_dmmatrix= xgb.DMatrix(data=Xtrain,label=Ytrain)

Loading word vectors...
Found 400000 word vectors.
Numer of samples with no words found: 0 / 6532


In [7]:
param = {
    'max_depth': 10,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 52} 
epochs = 1000

model = xgb.train(param, data_dmmatrix, epochs)

In [21]:
Xtest = vectorizer.transform(test.content)
test['label_idx'] = get_label(test['label'],trainLabelLst)
Ytest = test.label_idx
xgb_test = xgb.DMatrix(Xtest, label=Ytest)
predictions = model.predict(xgb_test)
accuracy_score(Ytest, predictions)

Numer of samples with no words found: 0 / 2568


0.8454049844236761

In [20]:
start_time = time.time()
model.predict(xgb_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.0065603256225586 seconds ---
